# Create Nutrition dataset
This dataset will be used to retrieve nutrition information for food category

In [5]:
import sys
!{sys.executable} -m pip install google-cloud-storage

In [40]:
from google.cloud import storage
import pandas as pd
import json
import numpy as np

## Retrieve data from Google cloud

In [16]:
BUCKET_NAME = 'foodygs'
bucket = client.bucket(BUCKET_NAME)
client = storage.Client()

Get the csv files from the Google Cloud storage

In [51]:
storage_filename_nut = 'Nutrition/nutrition.csv'
local_filename_nut = "../raw_data/nutrition.csv"

blob = bucket.blob(storage_filename_nut)
blob.download_to_filename(local_filename_nut)

Get the JSON file from the Google Cloud storage

In [49]:
storage_filename_annot = 'foodyai_data/Training_1/annotations.json'
local_filename_annot = "../raw_data/annotations.json"

blob = bucket.blob(storage_filename_annot)
blob.download_to_filename(local_filename_annot)

In [23]:
display(filename_nut)
display(local_filename_annot)

'nutrition.csv'

'annotations.json'

## Create dataframe 

### JSON file (annotations)

In [34]:
js = open(local_filename_annot)
ds_annot = json.load(js)

In [32]:
df_cat = pd.DataFrame(ds_annot['categories'])

In [33]:
df_cat.head()

,id,name,name_readable,supercategory
0,1565,bread-wholemeal,"Bread, wholemeal",food
1,2099,jam,Jam,food
2,2578,water,Water,food
3,1556,bread-sourdough,"Bread, sourdough",food
4,1154,banana,Banana,food


### CSV file (nutritions)

In [35]:
df_nut_1 = pd.read_csv(filename_nut)

In [36]:
df_nut_1.head()

,Unnamed: 0,name,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,...,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
0,0,Cornstarch,100 g,381,0.1g,NaN,0,9.00 mg,0.4 mg,0.00 mcg,...,0.05 g,0.009 g,0.016 g,0.025 g,0.00 mg,0.0 g,0.09 g,0.00 mg,0.00 mg,8.32 g
1,1,"Nuts, pecans",100 g,691,72g,6.2g,0,0.00 mg,40.5 mg,22.00 mcg,...,71.97 g,6.180 g,40.801 g,21.614 g,0.00 mg,0.0 g,1.49 g,0.00 mg,0.00 mg,3.52 g
2,2,"Eggplant, raw",100 g,25,0.2g,NaN,0,2.00 mg,6.9 mg,22.00 mcg,...,0.18 g,0.034 g,0.016 g,0.076 g,0.00 mg,0.0 g,0.66 g,0.00 mg,0.00 mg,92.30 g
3,3,"Teff, uncooked",100 g,367,2.4g,0.4g,0,12.00 mg,13.1 mg,0,...,2.38 g,0.449 g,0.589 g,1.071 g,0,0,2.37 g,0,0,8.82 g
4,4,"Sherbet, orange",100 g,144,2g,1.2g,1mg,46.00 mg,7.7 mg,4.00 mcg,...,2.00 g,1.160 g,0.530 g,0.080 g,1.00 mg,0.0 g,0.40 g,0.00 mg,0.00 mg,66.10 g


## Create new dataframe
Contains only category present in the JSON file

In [47]:
cat_csv = list(df_nut_1['name'])
cat_csv

['Cornstarch',
 'Nuts, pecans',
 'Eggplant, raw',
 'Teff, uncooked',
 'Sherbet, orange',
 'Cauliflower, raw',
 'Taro leaves, raw',
 'Lamb, raw, ground',
 'Cheese, camembert',
 'Vegetarian fillets',
 'PACE, Picante Sauce',
 'Goji berries, dried',
 'Mango nectar, canned',
 'Crackers, rusk toast',
 'Chicken, boiled, feet',
 'Quail, raw, meat only',
 'Pie, lemon, fried pies',
 'Peppers, raw, jalapeno',
 'Winged bean tuber, raw',
 'Salami, turkey, cooked',
 'Grapes, raw, muscadine',
 'Nuts, raw, ginkgo nuts',
 'Spices, ground, savory',
 'Candies, sesame crunch',
 'Cheese, low fat, cream',
 'PACE, Green Taco Sauce',
 'Syrup, Canadian, maple',
 'Ostrich, raw, top loin',
 'Chewing gum, sugarless',
 'Nuts, dried, pine nuts',
 'Pasta, unenriched, dry',
 "McDONALD'S, Side Salad",
 'Cookies, Marie biscuit',
 'Broccoli, raw, chinese',
 "McDONALD'S, Hash Brown",
 'Agave, raw (Southwest)',
 'Emu, raw, outside drum',
 'Nuts, dried, beechnuts',
 'Currants, dried, zante',
 'Lentils, raw, sprouted',
 'Gr

In [48]:
cat_json = list(df_cat['name_readable'])
cat_json

['Bread, wholemeal',
 'Jam',
 'Water',
 'Bread, sourdough',
 'Banana',
 'Soft cheese',
 'Ham, raw',
 'Hard cheese',
 'Cottage cheese',
 'Bread, half white',
 'Coffee, with caffeine',
 'Fruit salad',
 'Pancakes',
 'Tea',
 'Salmon, smoked',
 'Avocado',
 'Spring onion / scallion',
 'Ristretto, with caffeine',
 'Ham',
 'Egg',
 'Bacon, frying',
 'Chips, french fries',
 'Juice, apple',
 'Chicken',
 'Tomato, raw ',
 'Broccoli',
 'Shrimp, boiled',
 'Beetroot, steamed, without addition of salt',
 'Carrot, raw',
 'Chickpeas',
 'French salad dressing',
 'Pasta, Hörnli',
 'Sauce, cream',
 'Meat balls',
 'Pasta',
 'Tomato sauce',
 'Cheese',
 'Pear',
 'Cashew nut',
 'Almonds',
 'Lentils',
 'Mixed vegetables',
 'Peanut butter',
 'Apple',
 'Blueberries',
 'Cucumber',
 'Cocoa powder',
 'Greek Yaourt, yahourt, yogourt ou yoghourt',
 'Maple syrup (Concentrate)',
 'Buckwheat, grain peeled',
 'Butter',
 'Herbal tea',
 'Mayonnaise',
 'Soup, vegetable',
 'Wine, red',
 'Wine, white',
 'Green bean, steamed, wi